# Transit Service Areas - definition and summaries

Notebook is an implementation of https://github.com/BayAreaMetro/bayarea_urbansim/blob/main/scripts/proximity2transit.py. It will be turned into a standalone script.

In words, the process is going from a point layer of transit stops with information on size and headways and build status. Those are to be buffered, and a hierarchy is active, favoring rail over non-rail, and shorter headways over longer headways. Service areas a built by overlays, and parcels are classified by the service area they fall within. Finally model runs are summarized by these service areas, and other attributes.

# Paraphernalia

In [1]:
import os
import sys
import time
import traceback
import logging
import datetime

import pandas as pd
import geopandas as gpd
import fiona

/Users/aolsen/opt/anaconda3/envs/geo_env_2/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/4184460065.py:9: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Sha

In [551]:
# DVUTILS_LOCAL_CLONE_PATH = "/Users/aolsen/Documents/GitHub/dvutils"
# sys.path.insert(0, DVUTILS_LOCAL_CLONE_PATH)
# from utils_io import *

In [84]:
# gdb_path = '/Volumes/Data/Models/Scratch/AO/workspace_2024_0315_1017.gdb'
gdb_path = '/Volumes/Data/Models/Data/GIS layers/JobsHousingTransitProximity/workspace_2020_1008_2343.gdb'

#fiona.listlayers(gdb_path)
v2020_fbp_classes = gpd.read_file(gdb_path,layer='trn_bp_cat5')
v2020_cur_classes = gpd.read_file(gdb_path,layer='trn_cur_cat5')
v2020_np_classes = gpd.read_file(gdb_path,layer='trn_np_cat5')

In [ ]:
v2020_fbp_classes = gpd.read_file(gdb_path,layer='trn_bp_cat5')
v2020_cur_classes = gpd.read_file(gdb_path,layer='trn_cur_cat5')
v2020_np_classes = gpd.read_file(gdb_path,layer='trn_np_cat5')

In [264]:
fiona.listlayers(gdb_path)
v2020_transit_current = gpd.read_file(gdb_path,layer='transit_current')
v2020_transit_potential = gpd.read_file(gdb_path,layer='transit_potential')
v2020_transit_cur_major = gpd.read_file(gdb_path,layer='trn_cur_major')

In [102]:
# transit_areas_current_dissolved.explore(column='cat5',cmap='Set3')

In [101]:
# v2020_fbp_classes.explore(column='Service_Level',cmap='Set3')

## Paths
### AGOL URLs, and local versions until we get utils working

In [9]:
ANALYSIS_CRS = 'EPSG:26910'

In [10]:
WORKING_DIR = os.path.abspath(".")

In [386]:
MTC_ONLINE_TRANSIT_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/transitstops_existing_planned_2021/FeatureServer/0'

MTC_ONLINE_BACOUNTY_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/region_county/FeatureServer/0'

MTC_ONLINE_TAZ_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/transportation_analysis_zones_1454/FeatureServer/0'

#MTC_ONLINE_TRACT_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/Bay_Area_Census_Tracts_2010/FeatureServer/0'
MTC_ONLINE_TRACT_URL = 'https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/cocs_ACS2014_2018/FeatureServer/0'

In [560]:
tract_epc_path = '/Users/aolsen/Dropbox/My Mac (AOLSEN-MBP.local)/Downloads/proximity/communities_of_concern_2020_acs2018_-5728172521147435275.gpkg'
tract_epc21_path = '/Volumes/Data/Models/Data/Equity Priority Communities/PBA50/EPCs_ACS2018_tbl.csv'
tract_epc24_path = '/Volumes/Data/Models/Data/Equity Priority Communities/PBA50Plus/epc_acs2022.csv'
county_path = '/Users/aolsen/Dropbox/My Mac (AOLSEN-MBP.local)/Downloads/proximity/region_county_-6301486166122500334.gpkg'
transit_path = '/Users/aolsen/Dropbox/My Mac (AOLSEN-MBP.local)/Downloads/proximity/transitstops_existing_planned_2021_2777787894694541529.gpkg'
taz_path = '/Users/aolsen/Dropbox/My Mac (AOLSEN-MBP.local)/Downloads/proximity/transportation_analysis_zones_1454_5961535026220395633.gpkg'

In [578]:
tract_hra_2023_path = '/Volumes/Data/Models/Data/HCD/final_2023_public.xlsx'
tract_hra_2023 = pd.read_excel(tract_hra_2023_path,'BayArea',dtype={'Census Tract':str}).set_index('Census Tract')['Final Category']

In [583]:
tract_hra_2019_path = '/Volumes/Data/Models/Data/HCD/final-opportunity-map-statewide-summary-table.xlsx'
tract_hra_2019 = pd.read_excel(tract_hra_2019_path, 'BayArea', dtype={
                               'Tract FIPS Code': str}).set_index('Tract FIPS Code')['Final Category']
tract_hra_2019 = tract_hra_2019.fillna('Missing')

In [561]:
tract_epc21_file = pd.read_csv(tract_epc21_path)
tract_epc24_file = pd.read_csv(tract_epc24_path)

In [ ]:
county_file = gpd.read_file(county_path)
county_file = county_file.to_crs(ANALYSIS_CRS)

In [ ]:
transit_file = gpd.read_file(transit_path)
transit_file = transit_file.to_crs(ANALYSIS_CRS)

In [393]:
taz_file = gpd.read_file(taz_path)
taz_file = taz_file.to_crs(ANALYSIS_CRS)

In [562]:
# Enforce 11-character tract GEOID
tract_epc21_file['geoid10'] = tract_epc21_file.geoid.map(lambda x: f'{x:011d}')
tract_epc24_file['geoid20'] = tract_epc24_file.tract_geoid.map(lambda x: f'{x:011d}')

### Crosswalks

In [567]:
# set up crosswalks for parcels to something else
parcel_x_taz_file = '/Volumes/Data/Models/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/2020_08_17_parcel_to_taz1454sub.csv'
parcel_x_tract10_file = '/Volumes/Data/Models/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/parcel_tract_crosswalk.csv'
parcel_x_tracts_file = '/Volumes/Data/Models/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/p10_census.csv'
parcel_topo_file = '/Users/aolsen/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Policies/Base zoning/inputs/p10_geo.feather'
taz_x_areatype_file = '/Volumes/Data/Models/Application/Model One/RTP2021/Blueprint/INPUT_DEVELOPMENT/metrics/metrics_FinalBlueprint/taz_urban_suburban.csv'

In [16]:
parcel_x_taz = pd.read_csv(parcel_x_taz_file,
                           usecols=['PARCEL_ID', 'ZONE_ID'])

In [17]:
parcel_x_tract10 = pd.read_csv(parcel_x_tract10_file,
                               usecols=[
                                   'parcel_id', 'zone_id', 'GEOID10'])

In [573]:
taz_x_areatype = pd.read_csv(taz_x_areatype_file,index_col=['TAZ1454']).area_type


In [18]:
parcel_x_tracts = pd.read_csv(parcel_x_tracts_file)

In [22]:
# load p10 parcels

p10_topofix = gpd.read_feather(parcel_topo_file)
p10_topofix['geom_pt'] = p10_topofix.representative_point()
p10_topofix_pt = p10_topofix.set_geometry('geom_pt')[['PARCEL_ID', 'geom_pt']]

In [23]:
region_bdry = gpd.GeoDataFrame(data={'name':[0]},geometry=county_file.dissolve()['geometry'])
region_bdry.index = region_bdry.index.set_names('rowid')
region_bdry

,name,geometry
rowid,,
0,0,"MULTIPOLYGON (((634946.795 4149225.520, 634876..."


## Constants and mappings

In [29]:
KM_TO_MILES = 1.609344

# get the inverse
MILES_TO_KM = 1/KM_TO_MILES

In [30]:
def miles_to_m(miles):
    return 1000 * miles / MILES_TO_KM


# test: half mile to meters: should be ~805
miles_to_m(.5)

804.672

In [31]:
def get_combination_names(group):

    combination_names = []
    for col_name, value in group.iteritems():
        if value.any():
            # if there are any true values - keep the corresponding column name
            combination_names.append(col_name)
    return '-'.join(combination_names)

In [32]:
def get_combination_names_hierarchy(group):
    
    # loop through values in tuple - ordering matters. So
    # we stop at the first case of a True value - the 
    # first one takes primacy - like major stops buffer
    for col, val in group.iteritems():
        # print(col,val)
        if val.all():
            return f'{col}_only' if col!='reg' else 'none'

    return group

In [33]:
def subset_and_buffer(source_df, qry_str, buf_dist):
    source_subset = source_df.query(qry_str)

    source_subset_buf = source_subset.buffer(miles_to_m(buf_dist))

    output = (gpd.GeoDataFrame(  # data=source_subset[passthrough_cols],
        geometry=source_subset_buf.values)
        .reset_index()
        .rename(columns={'index': 'rowid'}))

    return output.dissolve()

# Create buffer for transit layer

## Set up logger

In [228]:
# create logger
NOW = datetime.datetime.now()
LOG_FILE = os.path.join(WORKING_DIR, "proximity2transit_{}.log".format(NOW))
logger = logging.getLogger(__name__)
logger.setLevel('DEBUG')

# console handler
ch = logging.StreamHandler()
ch.setLevel('INFO')
ch.setFormatter(logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p'))
logger.addHandler(ch)

# file handler
fh = logging.FileHandler(LOG_FILE, mode='w')
fh.setLevel('DEBUG')
fh.setFormatter(logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p'))
logger.addHandler(fh)

## Step 0: Subset transit data to relevant universes

In [280]:
# we use status criteria OR major_stop here -
# following logic here: https://github.com/BayAreaMetro/petrale/blob/c4b96a98b291ada58e065375beccd5bf11e2da1b/scripts/proximity2transit.py#L128

# subset to existing, under construction, open or FBP
# this is really just everything

# | major_stop==1
qry_fbp = 'status.isin(["Under Construction","Open","Final Blueprint","Existing/Built"]) '

transit_fbp = transit_file.query(
    qry_fbp)

# subset to existing major stop, and under construction or open
# | (major_stop==1 & status!="Final Blueprint")
qry_np = 'status.isin(["Under Construction","Open","Existing/Built"]) '

transit_np = transit_file.query(
    qry_np)

# subset to existing transit infrastructure (per 2015 or so, pre-plan)

qry_current = ' status=="Existing/Built"'
#qry_current = 'major_stop==1 '
transit_current = transit_file.query(
    qry_current)

print(transit_np.shape)
print(transit_fbp.shape)
print(transit_current.shape)

(40237, 18)
(40577, 18)
(40232, 18)


In [283]:
transit_file.query(qry_current).status.value_counts()

Existing/Built    40232
Name: status, dtype: int64

In [284]:
transit_file.query(qry_np).status.value_counts()

Existing/Built        40232
Under Construction        5
Name: status, dtype: int64

In [285]:
transit_file.query(qry_fbp).status.value_counts()

Existing/Built        40232
Final Blueprint         340
Under Construction        5
Name: status, dtype: int64

## Step 1: create transit buffers

In [286]:
def create_multiple_transit_areas(transit_stop_gdf, scenario='current', write_to_disk=False):

    logger.info(
        f"Running create_transit_areas function for variant: {scenario}")
    logger.info(f"transit_stop_gdf has shape: {transit_stop_gdf.shape}")

    buf_dist_half = .5

    qry_str_major = "(major_stop == 1)"
    logger.info(
        f"Subsetting stops to {qry_str_major}, and adding {buf_dist_half} mile buffer")
    major_buf = subset_and_buffer(
        transit_stop_gdf, qry_str_major, buf_dist_half)

    buf_dist_qtr = .25

    qry_str_hdwy_lt15 = "(am_av_hdwy <= 15) & (pm_av_hdwy <= 15)"
    logger.info(
        f"Subsetting stops to {qry_str_hdwy_lt15}, and adding {buf_dist_qtr} mile buffer")
    hdwy15_buf = subset_and_buffer(
        transit_stop_gdf, qry_str_hdwy_lt15, buf_dist_qtr)

    qry_str_hdwy_15_to_30 = "( am_av_hdwy >15 ) & (am_av_hdwy <=30) & ( pm_av_hdwy >15 ) & (pm_av_hdwy <=30)  "
    logger.info(
        f"Subsetting stops to {qry_str_hdwy_15_to_30}, and adding {buf_dist_qtr} mile buffer")
    hdwy30_buf = subset_and_buffer(
        transit_stop_gdf, qry_str_hdwy_15_to_30, buf_dist_qtr)

    qry_str_hdwy_gt30 = "(am_av_hdwy > 30) | (pm_av_hdwy > 30)"
    logger.info(
        f"Subsetting stops to {qry_str_hdwy_gt30}, and adding {buf_dist_qtr} mile buffer")
    hdwy30plus_buf = subset_and_buffer(
        transit_stop_gdf, qry_str_hdwy_gt30, buf_dist_qtr)

    ## Union and classify

    # these are the frames we are interested in for the union. We will denote in the
    # union process whether a geometry is covered by a particular spatial frame

    # the ordering here matters - reflecting a hierarchy of interest: we will use
    # to classify areas based on the highest order category in this list
    source_frames_ordered = ['major_buf', 'hdwy15buf', 'hdwy30buf', 'hdwy30plusbuf',
                             'reg']

    # geopandas can only handle two input layers at a time when unioning. We do it step-wise.

    logger.info(
        f"Beginning a series of unions, to get all buffers to fully define regional space")

    logger.info(f"Unioning hdwy15_buf and hdwy30_buf")

    # row_ids are in the source frames; we disambiguate to denote whether a particular
    # area is "covered" by either of the source files.
    # As we are dealing with dissolved geometries for the source geometries, it means
    # each source DF has just 1 row with multipart geoms - and that row_id then just
    # has the index value of 0 carried forward. So in the resulting union, a row_id value
    # of 0 means that a given area was represented in the relevant source dataframe;
    # a value of NaN means there was no corresponding match for a particular dataframe
    # this is useful for classifying areas based on the "source" buffer geometries present
    # for a particular location

    union_1 = gpd.overlay(hdwy15_buf, hdwy30_buf, how='union').reset_index().rename(
        columns={'index': 'df_1', 'rowid_1': 'hdwy15buf', 'rowid_2': 'hdwy30buf'})

    logger.info(f'union_1 shape: {union_1.shape}')

    logger.info(f"Adding hdwy30plus_buf...")
    union_2 = gpd.overlay(union_1, hdwy30plus_buf, how='union').reset_index().rename(
        columns={'index': 'df_2', 'rowid': 'hdwy30plusbuf'})

    # return union_1,hdwy15_buf, hdwy30_buf,union_2,hdwy30plus_buf

    logger.info(f'union_2 shape: {union_2.shape}')

    logger.info(f"Adding major stops buffer...")

    union_3 = gpd.overlay(union_2, major_buf, how='union').reset_index(
    ).rename(columns={'index': 'df_3', 'rowid': 'major_buf'})

    logger.info(f'union_3 shape: {union_3.shape}')
    logger.info(
        f"Adding regional boundary so we can get wall to wall coverage")

    # added small (here 10cm) geometry simplification to avoid a TopologyException: found non-noded intersection
    union_3['geometry'] = union_3.simplify(.1)

    union_4 = gpd.overlay(union_3, region_bdry, how='union').reset_index(
    ).rename(columns={'index': 'df_4', 'name': 'reg'})
    logger.info(f'union_4 shape: {union_4.shape}')
    logger.info('-'*80)
    logger.info(f'union_4 head: {union_4.head().T}')
    logger.info('-'*80)

    # except:
#     logger.error("Traceback info:\n{}\nError Info:\n{}".format(
#         'unions', str(sys.exc_info()[1])))

    # fill nas with -1 - those are values where a particular transit buffer layer doesn't cover something
    union_4 = union_4.fillna(-1)

    # we need boolean values to grab the group names
    logger.info(
        f"With the full region unioned, we now focus on classifying the different combinations")
    union_4[source_frames_ordered] = union_4[source_frames_ordered].replace(
        {0: True, -1: False})

    # get an integer encoding the unique combination of union components.
    # e.g. some areas will be in the buffer of a major stop and a 15 min headway
    # but not a 30 min headway. Others will only be in a 15 min headway buffer
    # we group areas that share the same combination. We then use that in the
    # naming of those groups just downstream; naming them based on
    # the boolean True values (indicating a particular buffer is represented)

    union_4['ngroup'] = union_4.groupby(source_frames_ordered).ngroup()

    # Then, for each combination of overlapping areas, get the most important
    # where rail (major stop) trumps 15min headway; 15min headway trumps 30 min headway etc

    union_4['cat5'] = union_4.ngroup.map(union_4.groupby(['ngroup'])[source_frames_ordered].apply(
        get_combination_names_hierarchy))

    # get the full string list of components in any given unioned area
    union_4['ngroup'] = union_4.ngroup.map(union_4.groupby(['ngroup'])[source_frames_ordered].apply(
        get_combination_names))

    if write_to_disk:
        union_4.to_file(f'/Users/aolsen/Downloads/union_4_{scenario}.geojson',driver='GeoJSON')

    return union_4, union_4.dissolve('cat5', as_index=False)[['cat5', 'geometry']]

In [287]:
# buf_dist_qtr = .25
# transit_stop_gdf = transit_current
# transit_stop_gdf = transit_fbp


# qry_str_hdwy_lt15 = "(am_av_hdwy <= 15) & (pm_av_hdwy <= 15)"
# logger.info(
#     f"Subsetting stops to {qry_str_hdwy_lt15}, and adding {buf_dist_qtr} mile buffer")
# hdwy15_buf_np = subset_and_buffer(
#     transit_current, qry_str_hdwy_lt15, buf_dist_qtr)

# hdwy15_buf_fbp = subset_and_buffer(
#     transit_fbp, qry_str_hdwy_lt15, buf_dist_qtr)

In [288]:
%%time 
transit_areas_current, transit_areas_current_dissolved = create_multiple_transit_areas(
    transit_current,'current')

03/19/2024 08:11:45 AM - INFO - Running create_transit_areas function for variant: current
03/19/2024 08:11:45 AM - INFO - Running create_transit_areas function for variant: current
03/19/2024 08:11:45 AM - INFO - transit_stop_gdf has shape: (40232, 18)
03/19/2024 08:11:45 AM - INFO - transit_stop_gdf has shape: (40232, 18)
03/19/2024 08:11:45 AM - INFO - Subsetting stops to (major_stop == 1), and adding 0.5 mile buffer
03/19/2024 08:11:45 AM - INFO - Subsetting stops to (major_stop == 1), and adding 0.5 mile buffer
03/19/2024 08:11:46 AM - INFO - Subsetting stops to (am_av_hdwy <= 15) & (pm_av_hdwy <= 15), and adding 0.25 mile buffer
03/19/2024 08:11:46 AM - INFO - Subsetting stops to (am_av_hdwy <= 15) & (pm_av_hdwy <= 15), and adding 0.25 mile buffer
03/19/2024 08:11:47 AM - INFO - Subsetting stops to ( am_av_hdwy >15 ) & (am_av_hdwy <=30) & ( pm_av_hdwy >15 ) & (pm_av_hdwy <=30)  , and adding 0.25 mile buffer
03/19/2024 08:11:47 AM - INFO - Subsetting stops to ( am_av_hdwy >15 ) & 

03/19/2024 08:15:55 AM - INFO - --------------------------------------------------------------------------------
03/19/2024 08:15:55 AM - INFO - --------------------------------------------------------------------------------
03/19/2024 08:15:55 AM - INFO - With the full region unioned, we now focus on classifying the different combinations
03/19/2024 08:15:55 AM - INFO - With the full region unioned, we now focus on classifying the different combinations
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/1584606566.py:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in group.iteritems():
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/1077116334.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, value in group.iteritems():


CPU times: user 4min 9s, sys: 965 ms, total: 4min 10s
Wall time: 4min 11s


In [289]:
%%time 
transit_areas_np, transit_areas_np_dissolved = create_multiple_transit_areas(
    transit_np,'np')

03/19/2024 08:15:57 AM - INFO - Running create_transit_areas function for variant: np
03/19/2024 08:15:57 AM - INFO - Running create_transit_areas function for variant: np
03/19/2024 08:15:57 AM - INFO - transit_stop_gdf has shape: (40237, 18)
03/19/2024 08:15:57 AM - INFO - transit_stop_gdf has shape: (40237, 18)
03/19/2024 08:15:57 AM - INFO - Subsetting stops to (major_stop == 1), and adding 0.5 mile buffer
03/19/2024 08:15:57 AM - INFO - Subsetting stops to (major_stop == 1), and adding 0.5 mile buffer
03/19/2024 08:15:57 AM - INFO - Subsetting stops to (am_av_hdwy <= 15) & (pm_av_hdwy <= 15), and adding 0.25 mile buffer
03/19/2024 08:15:57 AM - INFO - Subsetting stops to (am_av_hdwy <= 15) & (pm_av_hdwy <= 15), and adding 0.25 mile buffer
03/19/2024 08:15:59 AM - INFO - Subsetting stops to ( am_av_hdwy >15 ) & (am_av_hdwy <=30) & ( pm_av_hdwy >15 ) & (pm_av_hdwy <=30)  , and adding 0.25 mile buffer
03/19/2024 08:15:59 AM - INFO - Subsetting stops to ( am_av_hdwy >15 ) & (am_av_hdw

03/19/2024 08:20:04 AM - INFO - --------------------------------------------------------------------------------
03/19/2024 08:20:04 AM - INFO - --------------------------------------------------------------------------------
03/19/2024 08:20:04 AM - INFO - With the full region unioned, we now focus on classifying the different combinations
03/19/2024 08:20:04 AM - INFO - With the full region unioned, we now focus on classifying the different combinations
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/1584606566.py:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in group.iteritems():
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/1077116334.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, value in group.iteritems():


CPU times: user 4min 7s, sys: 570 ms, total: 4min 8s
Wall time: 4min 9s


In [290]:
%%time 
transit_areas_fbp, transit_areas_fbp_dissolved = create_multiple_transit_areas(
    transit_fbp,'fbp')

03/19/2024 08:20:06 AM - INFO - Running create_transit_areas function for variant: fbp
03/19/2024 08:20:06 AM - INFO - Running create_transit_areas function for variant: fbp
03/19/2024 08:20:06 AM - INFO - transit_stop_gdf has shape: (40577, 18)
03/19/2024 08:20:06 AM - INFO - transit_stop_gdf has shape: (40577, 18)
03/19/2024 08:20:06 AM - INFO - Subsetting stops to (major_stop == 1), and adding 0.5 mile buffer
03/19/2024 08:20:06 AM - INFO - Subsetting stops to (major_stop == 1), and adding 0.5 mile buffer
03/19/2024 08:20:07 AM - INFO - Subsetting stops to (am_av_hdwy <= 15) & (pm_av_hdwy <= 15), and adding 0.25 mile buffer
03/19/2024 08:20:07 AM - INFO - Subsetting stops to (am_av_hdwy <= 15) & (pm_av_hdwy <= 15), and adding 0.25 mile buffer
03/19/2024 08:20:08 AM - INFO - Subsetting stops to ( am_av_hdwy >15 ) & (am_av_hdwy <=30) & ( pm_av_hdwy >15 ) & (pm_av_hdwy <=30)  , and adding 0.25 mile buffer
03/19/2024 08:20:08 AM - INFO - Subsetting stops to ( am_av_hdwy >15 ) & (am_av_h

03/19/2024 08:24:48 AM - INFO - --------------------------------------------------------------------------------
03/19/2024 08:24:48 AM - INFO - --------------------------------------------------------------------------------
03/19/2024 08:24:48 AM - INFO - With the full region unioned, we now focus on classifying the different combinations
03/19/2024 08:24:48 AM - INFO - With the full region unioned, we now focus on classifying the different combinations
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/1584606566.py:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in group.iteritems():
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/1077116334.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, value in group.iteritems():


CPU times: user 4min 43s, sys: 442 ms, total: 4min 43s
Wall time: 4min 44s


In [548]:
transit_areas_fbp_dissolved.to_file('/Users/aolsen/Downloads/transit_areas_fbp_dissolved.geojson',driver='GeoJSON')

In [549]:
transit_areas_current_dissolved.to_file('/Users/aolsen/Downloads/transit_areas_current_dissolved.geojson',driver='GeoJSON')

In [536]:
mine_vs_bobbys_fbp = gpd.overlay(v2020_fbp_classes.to_crs(ANALYSIS_CRS),transit_areas_fbp_dissolved)

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/2050997278.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 17046 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  mine_vs_bobbys_fbp = gpd.overlay(v2020_fbp_classes.to_crs(ANALYSIS_CRS),transit_areas_fbp_dissolved)


In [ ]:
mine_vs_bobbys_fbp['area'] = mine_vs_bobbys_fbp.area

In [544]:
pct = lambda x: x/x.sum()
mine_vs_bobbys_fbp.groupby(['Service_Level','cat5']).area.sum().groupby(level=0,observed=True).apply(pct).round(2)

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_32079/615562494.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  mine_vs_bobbys_fbp.groupby(['Service_Level','cat5']).area.sum().groupby(level=0,observed=True).apply(pct).round(2)


Service_Level       cat5              
hdwy15buf_only      hdwy15buf_only        0.64
                    hdwy30buf_only        0.06
                    hdwy30plusbuf_only    0.03
                    major_buf_only        0.21
                    none                  0.06
hdwy30buf_only      hdwy15buf_only        0.03
                    hdwy30buf_only        0.70
                    hdwy30plusbuf_only    0.16
                    major_buf_only        0.07
                    none                  0.04
hdwy30plusbuf_only  hdwy15buf_only        0.00
                    hdwy30buf_only        0.03
                    hdwy30plusbuf_only    0.80
                    major_buf_only        0.02
                    none                  0.15
majorbuf            hdwy15buf_only        0.07
                    hdwy30buf_only        0.01
                    hdwy30plusbuf_only    0.02
                    major_buf_only        0.82
                    none                  0.09
none                h

In [416]:
# transit_areas_current.explore(column='cat5')

In [417]:
# transit_areas_fbp.explore(column='cat5')

## Step 2 - relate transit proximities to parcels (before adding any run specific data)

In [295]:
def classify_parcels(parcels_geo, transit_areas, scenario='current'):

    logger.info(f'Joining transit areas ({scenario}) to parcels')

    # core assignment, per sjoin
    p10_x_transit_areas = gpd.sjoin(parcels_geo, transit_areas)

    # check for unassigned
    unassigned_parcel_ids = set(
        parcels_geo.PARCEL_ID)-set(p10_x_transit_areas.PARCEL_ID)

    logger.info(
        f'There were {len(unassigned_parcel_ids)} parcels without an assignment to a transit area.\nWe assign to nearest area.')

    p10_x_transit_areas_remainder = gpd.sjoin_nearest(parcels_geo.query(
        'PARCEL_ID.isin(@unassigned_parcel_ids)'), transit_areas)

    # combine the two
    combo_assignments = pd.concat(
        [p10_x_transit_areas, p10_x_transit_areas_remainder])

    if (combo_assignments.PARCEL_ID.value_counts().sort_values() > 1).any():

        # if there are parcel duplicates - get the fist classification
        mapping = combo_assignments.groupby(
            'PARCEL_ID').cat5.first()
    else:
        # return the original mapping

        mapping = combo_assignments.set_index('PARCEL_ID').cat5

    return mapping

In [296]:
p10_x_transit_area_np = classify_parcels(p10_topofix_pt, transit_areas_np_dissolved, 'np')

03/19/2024 08:28:23 AM - INFO - Joining transit areas (np) to parcels
03/19/2024 08:28:23 AM - INFO - Joining transit areas (np) to parcels
03/19/2024 08:36:41 AM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/19/2024 08:36:41 AM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.


In [297]:
p10_x_transit_area_fbp = classify_parcels(p10_topofix_pt, transit_areas_fbp_dissolved, 'fbp')

03/19/2024 08:36:42 AM - INFO - Joining transit areas (fbp) to parcels
03/19/2024 08:36:42 AM - INFO - Joining transit areas (fbp) to parcels
03/19/2024 08:45:21 AM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/19/2024 08:45:21 AM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.


In [298]:
p10_x_transit_area_current = classify_parcels(p10_topofix_pt, transit_areas_current_dissolved, 'current')

03/19/2024 08:45:22 AM - INFO - Joining transit areas (current) to parcels
03/19/2024 08:45:22 AM - INFO - Joining transit areas (current) to parcels
03/19/2024 08:54:08 AM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.
03/19/2024 08:54:08 AM - INFO - There were 72 parcels without an assignment to a transit area.
We assign to nearest area.


In [545]:
p10_x_transit_area_np.value_counts()

none                  750973
major_buf_only        420949
hdwy30plusbuf_only    344821
hdwy30buf_only        286332
hdwy15buf_only        153132
Name: cat5, dtype: int64

In [546]:
p10_x_transit_area_fbp.value_counts()

none                  728135
major_buf_only        504425
hdwy30plusbuf_only    334198
hdwy30buf_only        263644
hdwy15buf_only        125805
Name: cat5, dtype: int64

In [547]:
p10_x_transit_area_current.value_counts()

none                  750984
major_buf_only        419789
hdwy30plusbuf_only    345460
hdwy30buf_only        286842
hdwy15buf_only        153132
Name: cat5, dtype: int64

## Step 3 load run data and classify based on parcel_id


## Step 4: run the summaries
Note that we want to do the proper matching:
* 2015 should always be "current"
* 2050 blueprint should be matched with 2050 blueprint transit buffers
* 2050 no project should be matched with 2050 no project transit buffers

We are looking for output as follows [metrics_proximity.csv](https://mtcdrive.app.box.com/file/837349294434)


In [433]:
metrics_proximity_target_schema = pd.read_csv(
    '/Users/aolsen/Box/Horizon and Plan Bay Area 2050/Equity and Performance/7_Analysis/Metrics/Metrics_Outputs_FinalBlueprint/Intermediate Metrics/Archive Draft Plan Mar 2021/metrics_proximity.csv')
schema_iteration_vars = ['Service_Level', 'year',
                         'modelrunID', 'transit', 'area', 'blueprint']
# metrics_proximity_target_schema.groupby(schema_iteration_vars).size().unstack('modelrunID')

In [461]:
transit_scenario_crosswalk = {'trn_cur_cat5': p10_x_transit_area_current,
                              'trn_fp_cat5': p10_x_transit_area_fbp,
                              'trn_np_cat5': p10_x_transit_area_np}

In [553]:
def col_values(s, colname):
    #msg = f'values for {colname}\n n{s.value_counts()}'
    msg2 = f'{colname} records with no value assigned: {len(s[s.isna()])}'
    #logger.info(msg)
    logger.info(msg2)
    
col_values(parcel_x_tracts.tract10,'trt')

03/19/2024 09:41:06 PM - INFO - trt records with no value assigned: 17
03/19/2024 09:41:06 PM - INFO - trt records with no value assigned: 17


In [587]:
def summarize_parcels(year, transit_scenario):

    #parcel_output = pd.read_csv(parcel_file, engine='python')

    logger.info('  Read {} rows'.format(len(parcel_output)))

    # keep essential columns
#     parcel_output.drop(['geom_id', 'total_job_spaces', 'zoned_du',
#                         'zoned_du_underbuild', 'zoned_du_underbuild_nodev', 'first_building_type'], axis=1, inplace=True)
    logger.info("Head:\n{}".format(parcel_output.head()))

    val_cols = ['totemp', 'RETEMPN', 'MWTEMPN', 'tothh']

    for col in val_cols:
        parcel_output[col] = parcel_output[col].fillna(0).round(0).astype(int)

    ####################################################
    # add tractids, tazs to parcels

    # vintage 2010
    parcel_output['tract10'] = parcel_output.parcel_id.map(
        parcel_x_tracts.set_index('parcel_id').tract10.map(lambda x: f'{x:011.0f}'))
    col_values(parcel_output['tract10'], 'tract10')

    # vintage 2020
    parcel_output['tract20'] = parcel_output.parcel_id.map(
        parcel_x_tracts.set_index('parcel_id').tract20.map(lambda x: f'{x:011.0f}'))
    col_values(parcel_output['tract20'], 'tract20')

    # add TAZs
    parcel_output['taz'] = parcel_output.parcel_id.map(
        parcel_x_taz.set_index('PARCEL_ID').ZONE_ID)
    col_values(parcel_output['taz'], 'taz')

    ####################################################

    # add area type based on the just assigned taz
    parcel_output['area_type'] = parcel_output.taz.map(taz_x_areatype)
    col_values(parcel_output['area_type'], 'area_type')

    # add epc (RTP2021 version) to parcels using tract10
    parcel_output['epc21'] = parcel_output.tract10.map(
        tract_epc21_file.set_index('geoid10').epc_class.fillna('Not EPC'))
    col_values(parcel_output['epc21'], 'epc21')

    parcel_output['is_epc21'] = parcel_output.epc21 != 'Not EPC'
    col_values(parcel_output['is_epc21'], 'is_epc21')

    # add epc (RTP2025 version) to parcels using tract10
    parcel_output['epc24'] = parcel_output.tract20.map(
        tract_epc24_file.set_index('geoid20').epc_class.fillna('Not EPC'))
    col_values(parcel_output['epc24'], 'epc24')

    parcel_output['is_epc24'] = parcel_output.epc24 != 'Not EPC'
    col_values(parcel_output['is_epc24'], 'is_epc24')

    # add hra23 to parcels using tract10
    parcel_output['hra23'] = parcel_output.tract20.map(tract_hra_2023)
    col_values(parcel_output['hra23'], 'hra23')

    parcel_output['is_hra23'] = parcel_output.hra23.isin(
        ["High Resource", "Highest Resource"])
    col_values(parcel_output['is_hra23'], 'is_hra23')

    # add hra19 to parcels using tract10
    parcel_output['hra19'] = parcel_output.tract20.map(tract_hra_2023)
    col_values(parcel_output['hra19'], 'hra19')

    parcel_output['is_hra19'] = parcel_output.hra19.isin(
        ["High Resource", "Highest Resource"])
    col_values(parcel_output['is_hra19'], 'is_hra19')

    # map the cat5 service level names to more descriptive ones used in Tableau
    parcel_output['service_level'] = parcel_output.parcel_id.map(
        transit_scenario_crosswalk[transit_scenario])
    col_values(parcel_output['service_level'], 'service_level')

    service_level_map = {'none': 'No_Fixed_Route_Transit',
                         'hdwy30plusbuf_only': 'Bus_31plus_min',
                         'hdwy30buf_only': 'Bus_15_30min',
                         'hdwy15buf_only': 'Bus_<15min',
                         'major_buf_only': 'Major_Transit_Stop'}

    parcel_output['Service_Level'] = parcel_output['service_level'].map(
        service_level_map)
    col_values(parcel_output['Service_Level'], 'Service_Level')

    def groupby_summaries(df, group_vars):

        grp_summary = (df
                       .groupby(group_vars)
                       .agg({'tothh': 'sum', 'hhq1': 'sum',
                             'totemp': 'sum', 'RETEMPN': 'sum',
                             'MWTEMPN': 'sum'})
                       )

        grp_summary_shares = grp_summary.div(
            grp_summary.sum()).round(3)

        grp_summary_combo = pd.concat([grp_summary,
                                       grp_summary_shares.add_suffix('_share')
                                       ], axis=1
                                      )
        return grp_summary_combo

    container = {}

    summary_servicelevel = groupby_summaries(
        parcel_output, ['Service_Level'])

    summary_servicelevel_epc = groupby_summaries(
        parcel_output, ['is_epc24', 'Service_Level'])

    summary_servicelevel_hra = groupby_summaries(
        parcel_output, ['is_hra23', 'Service_Level'])

    summary_servicelevel_areatype = groupby_summaries(
        parcel_output, ['area_type', 'Service_Level'])

    container['Region'] = pd.concat([summary_servicelevel], keys=['reg'])
    container['CoCs'] = summary_servicelevel_epc
    container['HRAs'] = summary_servicelevel_hra
    container['area_type'] = summary_servicelevel_areatype

    return pd.concat(container)


parcel_output_summary = summarize_parcels(2050, 'trn_fp_cat5')

03/19/2024 10:58:53 PM - INFO -   Read 1956212 rows
03/19/2024 10:58:53 PM - INFO -   Read 1956212 rows
03/19/2024 10:58:53 PM - INFO - Head:
   Unnamed: 0  parcel_id           x          y                  juris  \
0           0     229116 -121.795620  37.655379              livermore   
1           1     244166 -121.713004  37.717277              livermore   
2           2     202378 -122.014199  37.655260                hayward   
3           3    2004420 -122.771868  38.727893  unincorporated_sonoma   
4           4     340332 -121.974508  37.546277                fremont   

          fbpchcat  hhq1  hhq2  hhq3  hhq4  ...           Service_Level  \
0   NANAHRADISNAin   NaN   NaN   NaN   NaN  ...  No_Fixed_Route_Transit   
1    NAtra3DISNAin   NaN   NaN   NaN   NaN  ...  No_Fixed_Route_Transit   
2       NANANANAin   2.0   5.0  10.0  16.0  ...          Bus_31plus_min   
3  NANAHRADISNAout   NaN   NaN   NaN   NaN  ...  No_Fixed_Route_Transit   
4   NANAHRADISNAin   NaN   NaN   NaN  

In [588]:
parcel_output_summary

tothh      hhq1   totemp  \
                   Service_Level                                        
Region    reg      Bus_15_30min             446593   58616.0   700044   
                   Bus_31plus_min           471834   51835.0   711485   
                   Bus_<15min               258283   63388.0   354708   
                   Major_Transit_Stop      1970302  748626.0  2780424   
                   No_Fixed_Route_Transit   894545   87186.0   860152   
CoCs      False    Bus_15_30min             367240   44644.0   601227   
                   Bus_31plus_min           403813   40009.0   639696   
                   Bus_<15min               207563   49315.0   276238   
                   Major_Transit_Stop      1454362  522000.0  1948118   
                   No_Fixed_Route_Transit   828164   73727.0   761579   
          True     Bus_15_30min              79353   13972.0    98817   
                   Bus_31plus_min            68021   11826.0    71789   
                   Bus_<15min                50720   14073.0    78470   
                   Major_Transit_Stop       515940  226626.0   832306   
                   No_Fixed_Route_Transit    66381   13459.0    98573   
HRAs      False    Bus_15_30min             259194   34040.0   466649   
                   Bus_31plus_min           354388   40994.0   554385   
                   Bus_<15min               149294   36733.0   237975   
                   Major_Transit_Stop      1438364  573839.0  2202281   
                   No_Fixed_Route_Transit   547863   61534.0   650844   
          True     Bus_15_30min             187399   24576.0   233395   
                   Bus_31plus_min           117446   10841.0   157100   
                   Bus_<15min               108989   26655.0   116733   
                   Major_Transit_Stop       531938  174787.0   578143   
                   No_Fixed_Route_Transit   346682   25652.0   209308   
area_type rural    Bus_15_30min              17318    1988.0    17945   
                   Bus_31plus_min            63400    6042.0    72234   
                   Bus_<15min                 4044     786.0     9187   
                   Major_Transit_Stop        19812   10865.0    17863   
                   No_Fixed_Route_Transit   204304   14105.0   185509   
          suburban Bus_15_30min             226352   23356.0   377923   
                   Bus_31plus_min           262956   23132.0   311539   
                   Bus_<15min                87872   18833.0   122746   
                   Major_Transit_Stop       284373  104890.0   275473   
                   No_Fixed_Route_Transit   478465   42933.0   380645   
          urban    Bus_15_30min             202923   33272.0   304176   
                   Bus_31plus_min           145478   22661.0   327712   
                   Bus_<15min               166367   43769.0   222775   
                   Major_Transit_Stop      1666117  632871.0  2487088   
                   No_Fixed_Route_Transit   211776   30148.0   293998   

                                           RETEMPN  MWTEMPN  tothh_share  \
                   Service_Level                                           
Region    reg      Bus_15_30min              71128    94200        0.111   
                   Bus_31plus_min            56763    98146        0.117   
                   Bus_<15min                46591    39294        0.064   
                   Major_Transit_Stop       221462   192787        0.488   
                   No_Fixed_Route_Transit    45611   183222        0.221   
CoCs      False    Bus_15_30min              57878    82734        0.091   
                   Bus_31plus_min            49172    87999        0.100   
                   Bus_<15min                39401    27932        0.051   
                   Major_Transit_Stop       164327   148944        0.360   
                   No_Fixed_Route_Transit    40895   152916        0.205   
          True     Bus_15_30min              13250    11466       

In [602]:
# copying in the code from the metrics script for reference - 
# it is mainly an accounting script in the sense that the outputs are really coming from Bobby's scripts
# and by extension this notebook's scripts - share of households, etc by different transit service geographes

def calculate_Connected1_proximity(runid, year, dbp, transitproximity_df, metrics_dict):

    metric_id = "C1"

    for area_type in ['Region', 'CoCs', 'HRAs', 'rural', 'suburban', 'urban']:
        # households
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_tothh_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                       & (transitproximity_df['year'] == int(year))
                                                                                                                                       & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                       & (transitproximity_df['area'] == area_type), 'tothh_share'].sum()
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_hhq1_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                      & (transitproximity_df['year'] == int(year))
                                                                                                                                      & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                      & (transitproximity_df['area'] == area_type), 'hhq1_share'].sum()
        # jobs
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_totemp_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                        & (transitproximity_df['year'] == int(year))
                                                                                                                                        & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                        & (transitproximity_df['area'] == area_type), 'totemp_share'].sum()
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_RETEMPNjobs_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                             & (transitproximity_df['year'] == int(year))
                                                                                                                                             & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                             & (transitproximity_df['area'] == area_type), 'RETEMPN_share'].sum()
        metrics_dict[runid, metric_id, 'transitproximity_majorstop_shareof_MWTEMPNjobs_%s' % area_type, year, dbp] = transitproximity_df.loc[(transitproximity_df['Service_Level'] == "Major_Transit_Stop")
                                                                                                                                             & (transitproximity_df['year'] == int(year)) & (transitproximity_df['blueprint'].str.contains(dbp))
                                                                                                                                             & (transitproximity_df['area'] == area_type), 'MWTEMPN_share'].sum()

In [601]:
metrics = pd.read_csv('/Users/aolsen/Box/Horizon and Plan Bay Area 2050/Equity and Performance/7_Analysis/Metrics/Metrics_Outputs_FinalBlueprint/metrics.csv')
metrics.loc[(metrics.modelrunID.str.contains("PlusCrossing"))&(metrics['name'].str.contains('transitprox'))]

,modelrunID,metric,name,year,blueprint,value
2640,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_tothh_Region,2050,Plus,0.49
2641,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_hhq1_Region,2050,Plus,0.74
2642,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_totemp_Region,2050,Plus,0.51
2643,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_RETEMPNjobs...,2050,Plus,0.5
2644,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_MWTEMPNjobs...,2050,Plus,0.32
2645,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_tothh_CoCs,2050,Plus,0.68
2646,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_hhq1_CoCs,2050,Plus,0.82
2647,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_totemp_CoCs,2050,Plus,0.66
2648,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_RETEMPNjobs...,2050,Plus,0.61
2649,2050_TM152_FBP_PlusCrossing_24,C1,transitproximity_majorstop_shareof_MWTEMPNjobs...,2050,Plus,0.4


In [528]:
metrics_proximity_target_schema.query('blueprint=="Plus" & year==2050 & transit=="trn_fp_cat5" & area=="Region"') #& area=="CoCs"

,Service_Level,tothh,hhq1,totemp,RETEMPN,MWTEMPN,tothh_share,hhq1_share,totemp_share,RETEMPN_share,MWTEMPN_share,year,modelrunID,transit,area,blueprint
165,Bus_15_30min,241445,30886,415937,36569,68977,0.06,0.03,0.08,0.08,0.11,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
166,Bus_31plus_min,281742,31131,379888,31432,60476,0.07,0.03,0.07,0.07,0.10,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
167,Bus_<15min,786845,127837,1083359,115284,114261,0.19,0.13,0.20,0.26,0.19,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
168,Major_Transit_Stop,1971306,749222,2785232,221557,194184,0.49,0.74,0.51,0.50,0.32,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus
169,No_Fixed_Route_Transit,761974,70889,744044,36717,169760,0.19,0.07,0.14,0.08,0.28,2050,Final Blueprint runs/Final Blueprint (s24)/BAU...,trn_fp_cat5,Region,Plus


In [308]:
def parcel_file_name(scenario, year):

    for model_year in (2015, 2050):
        if model_year == 2050:
            if us_runid == us_2050_FBP_Final:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}_UBI.csv'.format(model_year)
            else:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}.csv'.format(model_year)
        else:
            parcel_file = urbansim_runid + \
                '_parcel_data_{}.csv'.format(model_year)
    urbansim_runid = os.path.join(urbansim_run_location, us_runid)

    return urbansim_runid

In [ ]:
def load_pba_parcel_data(scenario):

    column_map = {'tothh_share', 'hhq1_share',
                  'totemp_share', 'RETEMPN_share', 'MWTEMPN_share'}

    for model_year in (2015, 2050):
        if model_year == 2050:
            if us_runid == us_2050_FBP_Final:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}_UBI.csv'.format(model_year)
            else:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}.csv'.format(model_year)
        else:
            parcel_file = urbansim_runid + \
                '_parcel_data_{}.csv'.format(model_year)
        logger.info('Reading {} parcel data from {}'.format(
            model_year, parcel_file))
        parcel_output = pd.read_csv(parcel_file, engine='python')

        logger.info('  Read {} rows'.format(len(parcel_output)))

        # keep essential columns
        parcel_output.drop(['geom_id', 'total_job_spaces', 'zoned_du',
                            'zoned_du_underbuild', 'zoned_du_underbuild_nodev', 'first_building_type'], axis=1, inplace=True)
        logger.info("Head:\n{}".format(parcel_output.head()))

        parcel_output['totemp'] = parcel_output['totemp'].fillna(0)
        parcel_output['totemp'] = parcel_output['totemp'].round(
            0).astype('int')
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].fillna(0)
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].round(
            0).astype('int')
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].fillna(0)
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].round(
            0).astype('int')

In [ ]:
['year','transit','modelrunID','tothh_share'  ,'hhq1_share'  ,'totemp_share' ,'RETEMPN_share','MWTEMPN_share','modelrunID']

        prox_sdf_taz['tothh_share'  ] = round(prox_sdf_taz.tothh  / prox_sdf_taz.tothh.sum(),  2)
        prox_sdf_taz['hhq1_share'   ] = round(prox_sdf_taz.hhq1   / prox_sdf_taz.hhq1.sum()  , 2)
        prox_sdf_taz['totemp_share' ] = round(prox_sdf_taz.totemp / prox_sdf_taz.totemp.sum(), 2)
        prox_sdf_taz['RETEMPN_share'] = round(prox_sdf_taz.RETEMPN/prox_sdf_taz.RETEMPN.sum(), 2)
        prox_sdf_taz['MWTEMPN_share'] = round(prox_sdf_taz.MWTEMPN/prox_sdf_taz.MWTEMPN.sum(), 2)
        prox_sdf_taz['year'] = str(model_year)
        prox_sdf_taz['modelrunID'] = us_runid
        prox_sdf_taz['transit'] = transit_feature

In [82]:
# Bring in urbansim results
all_prox = pd.DataFrame()
taz_prox = pd.DataFrame()

for us_runid in list_us_runid:
    logger.info("")
    logger.info("==== Processing UrbanSim run {} ====".format(us_runid))
    urbansim_runid = os.path.join(urbansim_run_location, us_runid)

    for model_year in (2015, 2050):
        if model_year == 2050:
            if us_runid == us_2050_FBP_Final:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}_UBI.csv'.format(model_year)
            else:
                parcel_file = urbansim_runid + \
                    '_parcel_data_{}.csv'.format(model_year)
        else:
            parcel_file = urbansim_runid + \
                '_parcel_data_{}.csv'.format(model_year)
        logger.info('Reading {} parcel data from {}'.format(
            model_year, parcel_file))
        parcel_output = pd.read_csv(parcel_file, engine='python')

        logger.info('  Read {} rows'.format(len(parcel_output)))

        # keep essential columns
        parcel_output.drop(['geom_id', 'total_job_spaces', 'zoned_du',
                            'zoned_du_underbuild', 'zoned_du_underbuild_nodev', 'first_building_type'], axis=1, inplace=True)
        logger.info("Head:\n{}".format(parcel_output.head()))

        parcel_output['totemp'] = parcel_output['totemp'].fillna(0)
        parcel_output['totemp'] = parcel_output['totemp'].round(
            0).astype('int')
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].fillna(0)
        parcel_output['RETEMPN'] = parcel_output['RETEMPN'].round(
            0).astype('int')
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].fillna(0)
        parcel_output['MWTEMPN'] = parcel_output['MWTEMPN'].round(
            0).astype('int')

        # save as table in gdb
#         parcel_table = os.path.join(arcpy.env.workspace, "parcel_table")
#         if arcpy.Exists(parcel_table): arcpy.management.Delete(parcel_table)

#         parcel_array = np.array(np.rec.fromrecords(parcel_output.values))
#         parcel_array.dtype.names = tuple(parcel_output.dtypes.index.tolist())
#         arcpy.da.NumPyArrayToTable(parcel_array, parcel_table)
#         logger.info("Saved to {} in {}".format(parcel_table, arcpy.env.workspace))

        # convert to point feature class in GDB
#         if arcpy.Exists('parcel_fc'): arcpy.management.Delete('parcel_fc')
#         arcpy.management.XYTableToPoint(in_table=parcel_table, out_feature_class='parcel_fc',x_field='x',y_field='y')
#         logger.info("Saved to {} in {}".format('parcel_fc', arcpy.env.workspace))

03/18/2024 05:18:21 PM - INFO - 
03/18/2024 05:18:21 PM - INFO - ==== Processing UrbanSim run EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314 ====
03/18/2024 05:18:21 PM - INFO - Reading 2015 parcel data from /Users/aolsen/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/run314_parcel_data_2015.csv
03/18/2024 05:18:52 PM - INFO -   Read 1956212 rows
03/18/2024 05:18:52 PM - INFO - Head:
   parcel_id           x          y         fbpchcat                  juris  \
0     229116 -121.795620  37.655379   NANAHRADISNAin              livermore   
1     244166 -121.713004  37.717277    NAtra3DISNAin              livermore   
2     202378 -122.014199  37.655260       NANANANAin                hayward   
3    2004420 -122.771868  38.727893  NANAHRADISNAout  unincorporated_sonoma   
4     340332 -121.974508  37.546277   NANAHRADISNAin                fremont   

   hhq1  hhq2  hhq3  hhq4  tothh  ...  preserved_units  inclusionar

03/18/2024 05:20:48 PM - INFO -   Read 1956212 rows
03/18/2024 05:20:49 PM - INFO - Head:
   parcel_id          y           x                  juris  \
0     229116  37.655379 -121.795620              livermore   
1     244166  37.717277 -121.713004              livermore   
2     202378  37.655260 -122.014199                hayward   
3    2004420  38.727893 -122.771868  unincorporated_sonoma   
4     340332  37.546277 -121.974508                fremont   

                            eirzoningmodcat  hhq1  hhq2  hhq3  hhq4  tothh  \
0               livermoreNANAHRADISNANAinNA   NaN   NaN   NaN   NaN    NaN   
1                livermoreNAtra3DISNANAinNA   NaN   NaN   NaN   NaN    NaN   
2                    haywardNANANANANAinres   7.0   4.0  10.0  10.0   31.0   
3  unincorporated_sonomaNANAHRADISNANAoutNA   NaN   NaN   NaN   NaN    NaN   
4                 fremontNANAHRADISNANAinNA   NaN   NaN   NaN   NaN    NaN   

   ...  preserved_units  inclusionary_units  subsidized_units  AGREM

In [ ]:
if model_year == 2015:
        # current
        transit_features = ['trn_cur_cat5']
    elif model_year == 2050:
        # no plan and blueprint
        transit_features = ['trn_np_cat5', 'trn_fp_cat5']

    for transit_feature in transit_features:

        logger.info('Summarizing {} parcel data proximity to {}'.format(
            model_year, transit_feature))
        log_workspace_contents(logger)

        try:
            logger.info("feature classes no paths")
            arcpy.SummarizeWithin_analysis(transit_feature, 'parcel_fc', 'prox', keep_all_polygons="KEEP_ALL",
                                            sum_fields=[['tothh', 'SUM'], ['hhq1','SUM'],['totemp','SUM'],['RETEMPN','SUM'],['MWTEMPN','SUM']])
            # hasn't worked, see comments below
            logger.info("SUCCESS")
        except:
            # Get the tool error messages
            msgs = arcpy.GetMessages(2)
            logger.error("Exception occured; msgs: {}".format(msgs))

            # Get the traceback object
            tb = sys.exc_info()[2]
            tbinfo = traceback.format_tb(tb)[0]

            # Concatenate information together concerning the error into a message string
            logger.error("Traceback info:\n{}\nError Info:\n{}".format(
                tbinfo, str(sys.exc_info()[1])))
            logger.error(
                "It's ok though -- we'll do this another way, but still trying the easy way")

        # Something related to arcpy.SummarizeWithin_analysis() is buggy
        # The following attempts have failed with
        # ERROR 000187: Only supports Geodatabase tables and feature classes
        # * use the method with feature layers as inputs, with full paths and without
        # * use the method with feature classes as inputs, with full paths and without
        # * call the short script below via subprocess
        # * copy feature classes to arcpy.env.scratchGDB and summarizeWithin there
        #
        # HOWEVER, after this script fails, running the following on the command line succeeds:
        #
        # >>> import arcpy
        # >>> arcpy.env.workspace='M:\Data\GIS layers\JobsHousingTransitProximity\workspace_2020_1007_1737.gdb'
        # >>> arcpy.SummarizeWithin_analysis('trn_cur_cat5', 'parcel_fc', 'prox', keep_all_polygons='KEEP_ALL', sum_fields=[['tothh','SUM'], ['hhq1','SUM'],['totemp','SUM'],['RETEMPN','SUM'],['MWTEMPN','SUM']])
        # <Result 'M:\\Data\\GIS layers\\JobsHousingTransitProximity\\workspace_2020_1007_1737.gdb\\prox'>
        # >>> prox_sdf = pd.DataFrame.spatial.from_featureclass('prox')
        #
        # so we'll summarize within ourselves and use spatial join instead
        if arcpy.Exists('parcel_fc_join_trn'):
            arcpy.management.Delete('parcel_fc_join_trn')
        logger.info("spatial joining parcel_fc with {}".format(transit_feature))
        arcpy.SpatialJoin_analysis(target_features='parcel_fc', join_features=transit_feature,
                                   out_feature_class='parcel_fc_join_trn')

        logger.info("spatial joining parcel_fc_join_trn with {}".format(taz))
        arcpy.SpatialJoin_analysis(target_features='parcel_fc_join_trn', join_features=taz,
                                   out_feature_class='parcel_fc_join_trn_taz')

        logger.info(
            "spatial joining parcel_fc_join_trn_taz with {}".format(tract))
        arcpy.SpatialJoin_analysis(target_features='parcel_fc_join_trn_taz', join_features=tract,
                                   out_feature_class='parcel_fc_join_trn_taz_tract')
        logger.info("    ...complete")

        prox_sdf_ba = pd.DataFrame.spatial.from_featureclass(
            'parcel_fc_join_trn_taz_tract')
        prox_sdf = prox_sdf_ba.groupby('Service_Level').agg({'tothh': 'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf['tothh_share'  ] = round(prox_sdf.tothh  / prox_sdf.tothh.sum(),  2)
        prox_sdf['hhq1_share'   ] = round(prox_sdf.hhq1   / prox_sdf.hhq1.sum()  , 2)
        prox_sdf['totemp_share' ] = round(prox_sdf.totemp / prox_sdf.totemp.sum(), 2)
        prox_sdf['RETEMPN_share'] = round(prox_sdf.RETEMPN/prox_sdf.RETEMPN.sum(), 2)
        prox_sdf['MWTEMPN_share'] = round(prox_sdf.MWTEMPN/prox_sdf.MWTEMPN.sum(), 2)
        prox_sdf['year'] = str(model_year)
        prox_sdf['modelrunID'] = us_runid
        prox_sdf['transit'] = transit_feature
        prox_sdf['area'] = 'Region'

        logger.info("prox_sdf:\n{}".format(prox_sdf))
        all_prox = all_prox.append(prox_sdf)

        prox_sdf_epc = prox_sdf_ba[prox_sdf_ba.epc == 1]
        prox_sdf_epc = prox_sdf_epc.groupby('Service_Level').agg({'tothh': 'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf_epc['tothh_share'  ] = round(prox_sdf_epc.tothh  / prox_sdf_epc.tothh.sum(),  2)
        prox_sdf_epc['hhq1_share'   ] = round(prox_sdf_epc.hhq1   / prox_sdf_epc.hhq1.sum()  , 2)
        prox_sdf_epc['totemp_share' ] = round(prox_sdf_epc.totemp / prox_sdf_epc.totemp.sum(), 2)
        prox_sdf_epc['RETEMPN_share'] = round(prox_sdf_epc.RETEMPN/prox_sdf_epc.RETEMPN.sum(), 2)
        prox_sdf_epc['MWTEMPN_share'] = round(prox_sdf_epc.MWTEMPN/prox_sdf_epc.MWTEMPN.sum(), 2)
        prox_sdf_epc['year'] = str(model_year)
        prox_sdf_epc['modelrunID'] = us_runid
        prox_sdf_epc['transit'] = transit_feature
        prox_sdf_epc['area'] = 'epcs'

        logger.info("prox_sdf_epc:\n{}".format(prox_sdf_epc))
        all_prox = all_prox.append(prox_sdf_epc)

        prox_sdf_hra = prox_sdf_ba[prox_sdf_ba.hra == 1]
        prox_sdf_hra = prox_sdf_hra.groupby('Service_Level').agg({'tothh': 'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf_hra['tothh_share'  ] = round(prox_sdf_hra.tothh  / prox_sdf_hra.tothh.sum(),  2)
        prox_sdf_hra['hhq1_share'   ] = round(prox_sdf_hra.hhq1   / prox_sdf_hra.hhq1.sum()  , 2)
        prox_sdf_hra['totemp_share' ] = round(prox_sdf_hra.totemp / prox_sdf_hra.totemp.sum(), 2)
        prox_sdf_hra['RETEMPN_share'] = round(prox_sdf_hra.RETEMPN/prox_sdf_hra.RETEMPN.sum(), 2)
        prox_sdf_hra['MWTEMPN_share'] = round(prox_sdf_hra.MWTEMPN/prox_sdf_hra.MWTEMPN.sum(), 2)
        prox_sdf_hra['year'] = str(model_year)
        prox_sdf_hra['modelrunID'] = us_runid
        prox_sdf_hra['transit'] = transit_feature
        prox_sdf_hra['area'] = 'HRAs'

        logger.info("prox_sdf_hra:\n{}".format(prox_sdf_hra))
        all_prox = all_prox.append(prox_sdf_hra)

        logger.info("all_prox:\n{}".format(all_prox))

        prox_sdf_taz = prox_sdf_ba.groupby(['area_type', 'Service_Level']).agg({'tothh':'sum', 'hhq1':'sum', 
                                                         'totemp': 'sum', 'RETEMPN':'sum',
                                                         'MWTEMPN': 'sum'}).reset_index()

        prox_sdf_taz['tothh_share'  ] = round(prox_sdf_taz.tothh  / prox_sdf_taz.tothh.sum(),  2)
        prox_sdf_taz['hhq1_share'   ] = round(prox_sdf_taz.hhq1   / prox_sdf_taz.hhq1.sum()  , 2)
        prox_sdf_taz['totemp_share' ] = round(prox_sdf_taz.totemp / prox_sdf_taz.totemp.sum(), 2)
        prox_sdf_taz['RETEMPN_share'] = round(prox_sdf_taz.RETEMPN/prox_sdf_taz.RETEMPN.sum(), 2)
        prox_sdf_taz['MWTEMPN_share'] = round(prox_sdf_taz.MWTEMPN/prox_sdf_taz.MWTEMPN.sum(), 2)
        prox_sdf_taz['year'] = str(model_year)
        prox_sdf_taz['modelrunID'] = us_runid
        prox_sdf_taz['transit'] = transit_feature

        logger.info("prox_sdf:\n{}".format(prox_sdf_taz))
        taz_prox = taz_prox.append(prox_sdf_taz)

        logger.info("taz_prox:\n{}".format(taz_prox))

In [269]:
# union_1.to_file('/Users/aolsen/Downloads/union_1.geojson', driver='GeoJSON')
# union_2.to_file('/Users/aolsen/Downloads/union_2.geojson', driver='GeoJSON')
# union_3.to_file('/Users/aolsen/Downloads/union_3.geojson', driver='GeoJSON')
# union_4.to_file('/Users/aolsen/Downloads/union_4.geojson', driver='GeoJSON')

In [236]:
# hdwy15_buf.to_file(
#     '/Users/aolsen/Downloads/hdwy15_buf.geojson', driver='GeoJSON')
# hdwy30_buf.to_file(
#     '/Users/aolsen/Downloads/hdwy30_buf.geojson', driver='GeoJSON')
# hdwy30plus_buf.to_file(
#     '/Users/aolsen/Downloads/hdwy30plus_buf.geojson', driver='GeoJSON')
# major_buf.to_file(
#     '/Users/aolsen/Downloads/major_buf.geojson', driver='GeoJSON')

In [237]:

# diff_1 = gpd.overlay(hdwy15_buf,major_buf,how='difference')

# diff_2 = gpd.overlay(hdwy30_buf,major_buf,how='difference')
# diff_2b = gpd.overlay(diff_2,hdwy15_buf,how='difference')

# diff_3 = gpd.overlay(hdwy30plus_buf,major_buf,how='difference')
# diff_3b = gpd.overlay(diff_3,hdwy15_buf,how='difference')
# diff_3c = gpd.overlay(diff_3b,hdwy30_buf,how='difference')

In [ ]:
# check prefix+'_cat5'

In [ ]:
    logger.info("==== create_transit_features({}) ====".format(transit_type))
    if transit_type == "current":
        input_layer = "transit_current"
        prefix = "trn_cur"
    elif transit_type == "noplan":
        input_layer = "transit_potential"
        prefix = "trn_np"
        curprefix = "trn_cur"
    elif transit_type == "blueprint":
        input_layer = "transit_potential"
        prefix = "trn_fp"
        curprefix = "trn_cur"
    else:
        logger.fatal("Unsupported transit_type {}".format(transit_type))

In [113]:
transit_type_config = {
    "current": {
        "input_layer": "transit_current",
        "prefix": "trn_cur",
    },
    "noplan": {
        "input_layer": "transit_potential",
        "prefix": "trn_np",
        "curprefix": "trn_cur",
    },
    "blueprint": {
        "input_layer": "transit_potential",
        "prefix": "trn_fp",
        "curprefix": "trn_cur",
    },
}


In [7]:
def create_transit_features_new(logger, transit_type):
    """
    transit_type is one of [current, noplan, blueprint]
    """


    orig_result = arcpy.GetCount_management(input_layer)
    logger.info("{} has {} rows".format(input_layer, orig_result[0]))
    
    
    if transit_type == "current":
        logger.info('Select buffer for major transit stops => {}_majorbuf'.format(prefix))
        major = arcpy.SelectLayerByAttribute_management(input_layer, "NEW_SELECTION", '"major_stop" = 1')
        arcpy.CopyFeatures_management(major, prefix+"_major")  # save selection to new feature class
        major_result = arcpy.GetCount_management(prefix+"_major")
        logger.info("  {}_major has {} rows".format(prefix, major_result[0]))
        arcpy.Buffer_analysis(prefix+"_major", prefix+"_majorbuf", "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")
   
    else:
        if transit_type=="noplan":
            logger.info('Selecting "Under Construction" or "Open" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION", 
                                                                '"status" = \'Under Construction\' Or status=\'Open\'')
        elif transit_type=="blueprint":
            logger.info('Selecting "Under Construction" or "Open" or "Final Blueprint" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION", 
                                                                '"status" = \'Under Construction\' Or status=\'Open\' Or status=\'Final Blueprint\'')

        arcpy.CopyFeatures_management(new_major, prefix+"_new_major")  # save selection to new feature class
        new_major_result = arcpy.GetCount_management(prefix+"_new_major")
        logger.info("  {}_new_major has {} rows".format(prefix, new_major_result[0]))
        arcpy.Buffer_analysis(prefix+"_new_major", prefix+"_newmajorbuf", "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

        # merge new major buffered with original major buffered
        arcpy.Merge_management([curprefix+"_majorbuf",prefix+"_newmajorbuf"], prefix+"_majorbuf_predissolve")
        # dissolve
        arcpy.management.Dissolve(prefix+"_majorbuf_predissolve", prefix+"_majorbuf", 
                                  dissolve_field=None, statistics_fields=None, multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")


In [ ]:
def create_transit_features(logger, transit_type):
    """
    transit_type is one of [current, noplan, blueprint]
    """
    logger.info("==== create_transit_features({}) ====".format(transit_type))
    if transit_type == "current":
        input_layer = "transit_current"
        prefix = "trn_cur"
    elif transit_type == "noplan":
        input_layer = "transit_potential"
        prefix = "trn_np"
        curprefix = "trn_cur"
    elif transit_type == "blueprint":
        input_layer = "transit_potential"
        prefix = "trn_fp"
        curprefix = "trn_cur"
    else:
        logger.fatal("Unsupported transit_type {}".format(transit_type))

    orig_result = arcpy.GetCount_management(input_layer)
    logger.info("{} has {} rows".format(input_layer, orig_result[0]))

    # buffered transit_current major stops
    if transit_type == "current":
        logger.info(
            'Select buffer for major transit stops => {}_majorbuf'.format(prefix))
        major = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", '"major_stop" = 1')
        # save selection to new feature class
        arcpy.CopyFeatures_management(major, prefix+"_major")
        major_result = arcpy.GetCount_management(prefix+"_major")
        logger.info("  {}_major has {} rows".format(prefix, major_result[0]))
        arcpy.Buffer_analysis(prefix+"_major", prefix+"_majorbuf",
                              "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        if transit_type == "noplan":
            logger.info(
                'Selecting "Under Construction" or "Open" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION",
                                                                '"status" = \'Under Construction\' Or status=\'Open\'')
        elif transit_type == "blueprint":
            logger.info(
                'Selecting "Under Construction" or "Open" or "Final Blueprint" stops for no plan')
            new_major = arcpy.management.SelectLayerByAttribute(input_layer, "NEW_SELECTION",
                                                                '"status" = \'Under Construction\' Or status=\'Open\' Or status=\'Final Blueprint\'')

        # save selection to new feature class
        arcpy.CopyFeatures_management(new_major, prefix+"_new_major")
        new_major_result = arcpy.GetCount_management(prefix+"_new_major")
        logger.info("  {}_new_major has {} rows".format(
            prefix, new_major_result[0]))
        arcpy.Buffer_analysis(prefix+"_new_major", prefix+"_newmajorbuf",
                              "0.5 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

        # merge new major buffered with original major buffered
        arcpy.Merge_management(
            [curprefix+"_majorbuf", prefix+"_newmajorbuf"], prefix+"_majorbuf_predissolve")
        # dissolve
        arcpy.management.Dissolve(prefix+"_majorbuf_predissolve", prefix+"_majorbuf",
                                  dissolve_field=None, statistics_fields=None, multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")

    # buffered hdway_15min stops
    logger.info(
        'Creating buffer for stops with headway < 15min => {}_hdwy15buf'.format(prefix))

    if transit_type == "current":
        hdwy15 = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", "am_av_hdwy <= 15 And pm_av_hdwy <= 15")
        # save selection to new feature class
        arcpy.CopyFeatures_management(hdwy15, prefix+"_hdwy15")
        hdwy15_result = arcpy.GetCount_management(prefix+"_hdwy15")
        logger.info("  {}_hdwy15 has {} rows".format(prefix, hdwy15_result[0]))
        arcpy.Buffer_analysis(prefix+"_hdwy15", prefix+"_hdwy15buf",
                              "0.25 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        # copy current
        arcpy.CopyFeatures_management(
            curprefix+"_hdwy15buf", prefix+"_hdwy15buf")

    # buffered hdway_30min stops
    logger.info(
        'Creating buffer for stops with headway 15-30 min => {}_hdwy30buf'.format(prefix))

    if transit_type == "current":
        hdwy30 = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", "am_av_hdwy > 15 And am_av_hdwy <= 30 And pm_av_hdwy > 15 And pm_av_hdwy <= 30")
        # save selection to new feature class
        arcpy.CopyFeatures_management(hdwy30, prefix+"_hdwy30")
        hdwy30_result = arcpy.GetCount_management(prefix+"_hdwy30")
        logger.info("  {}_hdwy30 has {} rows".format(prefix, hdwy30_result[0]))
        arcpy.Buffer_analysis(prefix+"_hdwy30", prefix+"_hdwy30buf",
                              "0.25 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        # copy current
        arcpy.CopyFeatures_management(
            curprefix+"_hdwy30buf", prefix+"_hdwy30buf")

    # buffered hdway_30plus stops
    logger.info(
        'Creating buffer for stops with headway 30+ min => {}_hdwy30plusbuf'.format(prefix))

    if transit_type == "current":
        hdwy30plus = arcpy.SelectLayerByAttribute_management(
            input_layer, "NEW_SELECTION", "am_av_hdwy > 30 Or pm_av_hdwy > 30")
        # save selection to new feature class
        arcpy.CopyFeatures_management(hdwy30plus, prefix+"_hdwy30plus")
        hdwy30plus_result = arcpy.GetCount_management(prefix+"_hdwy30plus")
        logger.info("  {}_hdwy30plus has {} rows".format(
            prefix, hdwy30plus_result[0]))
        arcpy.Buffer_analysis(prefix+"_hdwy30plus", prefix+"_hdwy30plusbuf",
                              "0.25 Miles", "FULL", "ROUND", "ALL", None, "PLANAR")

    else:
        # copy current
        arcpy.CopyFeatures_management(
            curprefix+"_hdwy30plusbuf", prefix+"_hdwy30plusbuf")

    # Make them disjoint -- first one wins
    logger.info(
        'Isolate {}_hdwy15buf => {}_hdwy15buf_only'.format(prefix, prefix))
    arcpy.Erase_analysis(in_features=prefix+"_hdwy15buf", erase_features=prefix+"_majorbuf",
                         out_feature_class=prefix+"_hdwy15buf_only")

    logger.info(
        'Isolate {}_hdwy30buf => {}_hdwy30buf_only'.format(prefix, prefix))
    arcpy.Erase_analysis(prefix+"_hdwy30buf",   prefix +
                         "_majorbuf",  prefix+"_hdwy30buf_1")
    arcpy.Erase_analysis(prefix+"_hdwy30buf_1", prefix +
                         "_hdwy15buf", prefix+"_hdwy30buf_only")
    arcpy.Delete_management([prefix+"_hdwy30buf_1"])

    logger.info(
        'Isolate {}_hdwy30plusbuf  => {}_hdwy30plusbuf_only'.format(prefix, prefix))
    arcpy.Erase_analysis(prefix+"_hdwy30plusbuf",   prefix +
                         "_majorbuf",  prefix+"_hdwy30plusbuf_1")
    arcpy.Erase_analysis(prefix+"_hdwy30plusbuf_1", prefix +
                         "_hdwy15buf", prefix+"_hdwy30plusbuf_2")
    arcpy.Erase_analysis(prefix+"_hdwy30plusbuf_2", prefix +
                         "_hdwy30buf", prefix+"_hdwy30plusbuf_only")
    arcpy.Delete_management([prefix+"_hdwy30plusbuf_1",
                             prefix+"_hdwy30plusbuf_2"])

    logger.info('Rest of Bay Area => {}_none'.format(prefix))
    arcpy.Erase_analysis("BAcounty_expand",   prefix +
                         "_majorbuf",      "BAcounty_expand_1")
    arcpy.Erase_analysis("BAcounty_expand_1", prefix +
                         "_hdwy15buf",     "BAcounty_expand_2")
    arcpy.Erase_analysis("BAcounty_expand_2", prefix +
                         "_hdwy30buf",     "BAcounty_expand_3")
    arcpy.Erase_analysis("BAcounty_expand_3", prefix +
                         "_hdwy30plusbuf", prefix+"_none")
    arcpy.Delete_management(["BAcounty_expand_1",
                             "BAcounty_expand_2",
                             "BAcounty_expand_3"])

    logger.info('Merge into one feature class => {}_cat5'.format(prefix))
    arcpy.Merge_management([prefix+"_none",
                            prefix+"_hdwy30plusbuf_only",
                            prefix+"_hdwy30buf_only",
                            prefix+"_hdwy15buf_only",
                            prefix+"_majorbuf"],
                           prefix+"_cat5", add_source="ADD_SOURCE_INFO")
    # create Service_Level from MERGE_SRC
    arcpy.AddField_management(
        prefix + "_cat5", "Service_Level", "TEXT", "", "", 200)
    with arcpy.da.UpdateCursor(prefix + "_cat5", ["Service_Level", "MERGE_SRC"]) as cursor:
        for row in cursor:
            if 'none' in row[1]:
                row[0] = 'No_Fixed_Route_Transit'
            elif 'hdwy30plusbuf' in row[1]:
                row[0] = 'Bus_31plus_min'
            elif 'hdwy30buf' in row[1]:
                row[0] = 'Bus_15_30min'
            elif 'hdwy15buf' in row[1]:
                row[0] = 'Bus_<15min'
            elif 'majorbuf' in row[1]:
                row[0] = 'Major_Transit_Stop'
            cursor.updateRow(row)

In [6]:
import geopandas as gpd
from shapely.geometry import Point  # for creating point buffers


def create_transit_features(logger, transit_type):
    """
    transit_type is one of [current, noplan, blueprint]
    """
    logger.info("==== create_transit_features({}) ====".format(transit_type))

    # Input data loading (adjust path and format)
    # Replace with correct path and format
    input_gdf = gpd.read_file("path/to/your/data.shp")

    if transit_type == "current":
        prefix = "trn_cur"
        curprefix = "trn_cur"
    elif transit_type == "noplan":
        prefix = "trn_np"
        curprefix = "trn_cur"
    elif transit_type == "blueprint":
        prefix = "trn_fp"
        curprefix = "trn_cur"
    else:
        logger.fatal("Unsupported transit_type {}".format(transit_type))

    # Calculate feature counts (use describe method)
    feature_counts = input_gdf.describe()
    logger.info(f"{input_layer} has {feature_counts['geometry'].iloc[0]} rows")

    # Select features based on conditions (use query method)
    if transit_type == "current":
        major_gdf = input_gdf.query("major_stop == 1")
    else:
        if transit_type == "noplan":
            new_major_gdf = input_gdf.query(
              "(status == 'Under Construction') | (status == 'Open')"
          )
    else:
        new_major_gdf = input_gdf.query(
          "(status == 'Under Construction') | (status == 'Open') | (status == 'Final Blueprint')"
      )

    # Buffer features (use buffer method)
    majorbuf_gdf = major_gdf.buffer(distance=0.5, units="miles")
    if transit_type != "current":
        hdwy15buf_gdf = gpd.read_file(
            f"{curprefix}_hdwy15buf.shp")  # Assuming existing buffer
        hdwy30buf_gdf = gpd.read_file(
            f"{curprefix}_hdwy30buf.shp")  # Assuming existing buffer
        hdwy30plusbuf_gdf = gpd.read_file(
            f"{curprefix}_hdwy30plusbuf.shp")  # Assuming existing buffer

    # ... (similar logic for hdwy15, hdwy30, hdwy30plus selections and buffering using query and buffer methods)

    # Isolate buffers using difference (use difference method)
    if transit_type == "current":
        hdwy15buf_only_gdf = majorbuf_gdf.difference(hdwy30buf_gdf)
        hdwy30buf_only_gdf = (
            majorbuf_gdf.difference(
                hdwy15buf_gdf).difference(hdwy30plusbuf_gdf)
        )
        hdwy30plusbuf_only_gdf = majorbuf_gdf.difference(
            hdwy15buf_gdf).difference(hdwy30buf_gdf)
    else:
    # ... (similar logic for isolating buffers using difference for noplan and blueprint)
        pass

    # Isolate remaining area using difference (use difference method)
    none_gdf = gpd.read_file("BAcounty_expand.shp")  # Assuming existing data
    none_gdf = none_gdf.difference(majorbuf_gdf)
    if transit_type != "current":
        none_gdf = none_gdf.difference(hdwy15buf_gdf)
        none_gdf = none_gdf.difference(hdwy30buf_gdf)
        none_gdf = none_gdf.difference(hdwy30plusbuf_gdf)

    # Merge into one GeoDataFrame (use concat and GeoDataFrame constructor)
    merged_gdf = gpd.GeoDataFrame(pd.concat([
      none_gdf, hdwy30plusbuf_only_gdf, hdwy30buf_only_gdf,
      hdwy15

IndentationError: expected an indented block after 'else' statement on line 34 (3996946805.py, line 35)